In [1]:
# DELETE THIS CELL IN THE STUDENT NOTEBOOK
# Create the student notebook
!pip install gsexport -q
import gsExport
!jupyter nbconvert museums_module_instructor.ipynb --NotebookExporter.preprocessors="['gsExport.gsExportPreprocessor']" --to notebook

[NbConvertApp] Converting notebook museums_module_instructor.ipynb to notebook
[NbConvertApp] Writing 76965 bytes to museums_module_instructor.nbconvert.ipynb


# Natural History Museums and Biodiversity Science Data Science Module

The goal of this notebook is to access and integrate diverse data sets to visualize correlations and discover patterns to address questions of species’ responses to environmental change. We will use programmatic tools to show how to use Berkeley resources such as the biodiversity data from biocollections and online databases, field stations, climate models, and other environmental data. If you have any questions getting the Jupyter notebook to run, try dropping into [data peer consulting](https://data.berkeley.edu/education/data-peer-consulting).

Before we begin analyzing and visualizing biodiversity data, this introductory notebook will help familiarize you with the basics of programming in Python.

## Table of Contents

Please complete the pre-lab before running through this notebook. Sections 0 and 1 are identical to the pre-lab and are included here for your reference.

0 - [Jupyter Notebooks](#jupyter)
    
1 - [Python Basics](#python)

2 - [GBIF API](#gbif)

3 - [Mapping](#mapping)

4 - [Comparing California Oak Species](#oak)

5 - [Cal-Adapt](#caladapt)

# Part 0: Our Computing Environment, Jupyter notebooks  <a id='jupyter'></a>
This webpage is called a Jupyter notebook. A notebook is a place to write programs and view their results. 

### Text cells
In a notebook, each rectangle containing text or code is called a *cell*.

Text cells (like this one) can be edited by double-clicking on them. They're written in a simple format called [Markdown](http://daringfireball.net/projects/markdown/syntax) to add formatting and section headings.  You don't need to learn Markdown, but you might want to.

After you edit a text cell, click the "run cell" button at the top that looks like ▶| to confirm any changes. You can also hold `shift` and then press `enter` or `return`. (Try not to delete the instructions of the lab.)

### Code cells
Other cells contain code in the Python 3 language. Running a code cell will execute all of the code it contains.

To run the code in a code cell, first click on that cell to activate it.  It'll be highlighted with a little green or blue rectangle.  Next, either press ▶| or hold down the `shift` key and press `return` or `enter`.

Try running this cell:

In [ ]:
print("Hello, World!")

And this one:

In [ ]:
print("\N{WAVING HAND SIGN}, \N{EARTH GLOBE ASIA-AUSTRALIA}!")

In order to finish the setup for this notebook, run the following cell:

In [ ]:
%%capture
!pip install --no-cache-dir shapely
!pip install -U folium

%matplotlib inline

import pandas as pd
import numpy as np
import folium
import json

from shapely.geometry import Point, mapping
from shapely.geometry.polygon import Polygon
from shapely import geometry as sg, wkt
from scripts.espm_module import *
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt

plt.style.use('seaborn')

# Part 1: Python basics <a id='python'></a>
Before getting into the more high level analyses we will do on the GBIF and Cal-Adapt data, we need to cover a few of the foundational elements of programming in Python.

#### A. Expressions
The departure point for all programming is the concept of the __expression__. An expression is a combination of variables, operators, and other Python elements that the language interprets and acts upon. Expressions act as a set of instructions to be fed through the interpreter, with the goal of generating specific outcomes. See below for some examples of basic expressions. Keep in mind that most of these just map to your understanding of mathematical expressions:

In [ ]:
2 + 2

'me' + ' and I'

12 ** 2

6 + 4

You will notice that only the last line in a cell gets printed out. If you want to see the values of previous expressions, you need to call the `print` function on that expression. Functions use parenthesis around their parameters, just like in math!

In [ ]:
print(2 + 2)

print('you' + ' and I')

print(12 ** 2)

print(6 + 4)

#### B. Variables
In the example below, `a` and `b` are Python objects known as __variables__. We are giving an object (in this case, an `integer` and a `float`, two Python data types) a name that we can store for later use. To use that value, we can simply type the name that we stored the value as. Variables are stored within the notebook's environment, meaning stored variable values carry over from cell to cell.

In [ ]:
a = 4
b = 10/5

Notice that when you create a variable, unlike what you previously saw with the expressions, it does not print anything out.

We can continue to perform mathematical operations on these variables, which are now placeholders for what we've assigned:

In [ ]:
print(a + b)

#### C. Lists
The following few cells will introduce the concept of __lists__.

A list is an ordered collection of objects. They allow us to store and access groups of variables and other objects for easy access and analysis. Check out this [documentation](https://www.tutorialspoint.com/python/python_lists.htm) for an in-depth look at the capabilities of lists.

To initialize a list, you use brackets. Putting objects separated by commas in between the brackets will add them to the list. 

We use `lst` as the variable name because `list` is a reserved keyword in Python (meaning it has special functionality and shouldn't be used as a variable name).

In [ ]:
# an empty list
lst = []
print(lst)

# reassigning our empty list to a new list
lst = [1, 3, 6, 'lists', 'are' 'fun', 4]
print(lst)

To access a value in the list, put the index of the item you wish to access in brackets following the variable that stores the list. Lists in Python are zero-indexed, so the indicies for `lst` are 0, 1, 2, 3, 4, 5, and 6.

In [ ]:
# Elements are selected like this:
example = lst[2]

# The above line selects the 3rd element of lst (list indices are 0-offset) and sets it to a variable named example.
print(example)

#### D. Dictionaries

Dictionaries are `key`-`value` pairs. Just like a word dictinary, you have a key that will index a specific definition.

In [ ]:
my_dict = {'python': 'a large heavy-bodied nonvenomous constrictor snake occurring throughout the Old World tropics.'}

We can get a `value` back out by indexing the `key`:

In [ ]:
my_dict['python']

But like real dictionaries, there can be more than one definition. You can keep a `list`, or even another dictionary within a specific `key`:

In [ ]:
my_dict = {'python': ['a large heavy-bodied nonvenomous constrictor snake occurring throughout the Old World tropics.',
                      'a high-level general-purpose programming language.']}

We can index the `list` after the `key`:

In [ ]:
my_dict['python'][0]

In [ ]:
my_dict['python'][1]

**E: Functions**

In programming, we often reuse chunks of code. So instead of copy/pasting it and repeating the same code over and over again, we have something called a function, which gives a name to a block of code. This allows us to just call the function instead of rewriting code we used before.

For example, this is a function that squares an input.

In [ ]:
# This code creates a function named square
def square(n):
    return n * n

In [ ]:
# Let's find the square of 5
square(5)

In [ ]:
# Let's try it with -3
square(-3)

Our use of functions later in the notebook is more complex than this example. We will use them in order to reduce the amount of code in this notebook. For now, you can just ignore the details and structure of how functions work. Just remember that a **function** is a shortcut to easily re-run old code and that the `def` keyword means we are creating a function.

---

# Part 2: GBIF API<a id='gbif'></a>

Click on the [link](http://www.gbif.org/) to the GBIF website to discover what GBIF can do!

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 1:</b>
    <br />
    What does GBIF stand for and who is it coordinated by?
</div>

#student

YOUR RESPONSE HERE

The Global Biodiversity Information Facility has created an API that we can use to get data about different species at the [GBIF Web API](http://www.gbif.org/developer/summary).

You can think of a Web API call as a fancy URL. What do you think the end of this URL means?

http://api.gbif.org/v1/occurrence/search?year=1800,1899

If you're guessing that it limits the search to the years 1800-1899, you're right! Go ahead and click the URL above. You should see something like this:

```
{"offset":0,"limit":20,"endOfRecords":false,"count":5711947,"results":[{"key":14339704,"datasetKey":"857aa892-f762-11e1-a439-00145eb45e9a","publishingOrgKey":"6bcc0290-6e76-11db-bcd5-b8a03c50a862","publishingCountry":"FR","protocol":"BIOCASE","lastCrawled":"2013-09-07T07:06:34.000+0000","crawlId":1,"extensions":{},"basisOfRecord":"OBSERVATION","taxonKey":2809968,"kingdomKey":6,"phylumKey":7707728,"classKey":196,"orderKey":1169,"familyKey":7689,"genusKey":2849312,"speciesKey":2809968,"scientificName":"Orchis militaris L.","kingdom":"Plantae","phylum":"Tracheophyta","order":"Asparagales","family":"Orchidaceae","genus":"Orchis","species":"Orchis 
```

It might look like a mess, but it's not! This is actually very structured data, and can easily be put into a table like format, though often programmers don't do this because it's just as easy to keep it as is.

You might be able to pick out the curly braces `{` and think this it's a dictionary. You'd be right, except in this format we call it [JSON](https://en.wikipedia.org/wiki/JSON).

---

## *Argia arioides*

![argia arioides](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Argia_agrioides-Male-1.jpg/220px-Argia_agrioides-Male-1.jpg)

When performing data analysis, it is always important to define a question that you seek the answer to. *The goal of finding the answer to this question will ultimately drive the queries and analysis styles you choose to use/write.*

For this example, we are going to ask: **where have [*Argia agrioides*](https://www.google.com/search?q=Argia+agrioides&rlz=1C1CHBF_enUS734US734&source=lnms&tbm=isch&sa=X&ved=0ahUKEwji9t29kNTWAhVBymMKHWJ-ANcQ_AUICygC&biw=1536&bih=694) (the California Dancer dragonfly) been documented? Are there records at any of our field stations?**

The code to ask the API has already been written for us! This is often the case with programming, someone has already written the code, so we don't have to. We'll just set up the `GBIFRequest` object and assign that to the variable `req`, short for "request":

In [ ]:
req = GBIFRequest()  # creating a request to the API

Great, so how do we make searches? We can use a Python `dictionary` to create our query parameters. We'll ask for the `scientificName` of the California Dancer (*Argia arioides*):

In [ ]:
params = {'scientificName': 'Argia agrioides'}  # setting our parameters (the specific species we want)

Now that we have the parameters, we can feed this to our `req` variable to get back all the pages of data. We'll then make sure that each record has a `decimalLatitude`, otherwise we'll thow it out for now. Lastly, we'll print out the first five records:

In [ ]:
params = {'scientificName': 'Argia agrioides'}  # setting our parameters (the specific species we want)
pages = req.get_pages(params)  # using those parameters to complete the request
records = [rec for page in pages for rec in page['results'] if rec.get('decimalLatitude')]  # sift out valid records
records[:5]  # print first 5 records

That was a lot of code for something that we will be doing multiple times. Instead of copy/pasting that code when we want to get more species records, we can just create something called a function to get the records for us. For now, think of a function as a shortcut to re-run larger chunks of code as one command. The following code cell defines a function to get a given species' records. Notice how almost all of the indented code is the same code we ran above.

In [ ]:
def get_species_records(scientific_name):
    req = GBIFRequest()  # creating a request to the API
    params = {'scientificName': scientific_name}  # setting our parameters (the specific species we want)
    pages = req.get_pages(params)  # using those parameters to complete the request
    records = [rec for page in pages for rec in page['results'] if rec.get('decimalLatitude')]  # sift out valid records
    return records

Because we have defined a function, we can get the records of a species in just one line!

In [ ]:
records = get_species_records('Argia agrioides') # Get the species records using the function we defined
records[:5] # Show the first five rows of the records we obtained

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 2:</b>
    <br />
    Why might it be useful to know the documented occurences of a species? Name one organization which would find this information useful.
</div>

#student

YOUR RESPONSE HERE

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 3:</b>
    <br />
    What is the geographic range of an organism.
</div>

#student

YOUR RESPONSE HERE

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 4:</b>
    <br />
    How do museum records help us to understand how populations are changing?
</div>

#student

YOUR RESPONSE HERE

---

### DataFrames

The JSON-format data we looked at above contains helpful information but it's really hard to see look through. The text we saw above was just the first 5 entries! Instead, let's create a table that organizes this data for us. We'll use the popular [Pandas](http://pandas.pydata.org/) Python library. In `pandas`, a DataFrame is a table that has several convenient features that make it much easier to read, access, and analyze data.

In [ ]:
records_df = pd.read_json(json.dumps(records))  # converts the JSON above to a dataframe
records_df.head()                               # prints the first five rows of the dataframe

Since we're going to want to get DataFrames of species records several times in the future, let's also create function that does this for us.

In [ ]:
def get_species_records_df(scientific_name):
    records = get_species_records(scientific_name) # Get the records using the function we defined earlier
    records_df = pd.read_json(json.dumps(records)) # Convert the raw records into a DataFrame
    return records_df

In [ ]:
records_df = get_species_records_df('Argia agrioides')

DataFrames have a shape attribute which describes its row and column count.

In [ ]:
records_df.shape

So we have 302 rows and 115 columns! That's a lot of information. What variables do we have in the columns?

In [ ]:
list(records_df.columns) # Show a list containing the names of every column in our DataFrame

We can also use Pandas to analyze and visualize our data. The `value_counts()` function groups our data by a specified column and counts the number of occurrences of each column value.

In [ ]:
# How many records are there from every country
records_df['country'].value_counts()

This tells us that we're looking at data from the US and Mexico. For small examples, this is sufficient, but for columns with more unique values, it might be helpful to look at a bar graph instead. Pandas has a `bar` function that does that for us. Don't worry about the specific syntax of how we do this.

In the following series of graphps, you might notice that we switch between horizontal and vertical bar graphs. We are going to pick whichever one we think makes our graph easier to read. There isn't much of a difference beyond that.

In [ ]:
# Group the data by country and then plot it
records_df['country'].value_counts().plot.bar();

# This makes the graph easier to read
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by Country');
plt.xticks(rotation=0);
plt.xlabel('Country');
plt.ylabel('Count');

In [ ]:
# Code to quickly clean up county names
# You can ignore this
def clean_up_county(col):
    if type(col) == float and np.isnan(col):
        return None
    else:
        return col.replace(' Co.', '').replace(' County', '') + ' County'

# Use the function defined above to clean up county names
records_df['county_cleaned'] = records_df['county'].apply(clean_up_county)

# Make graph easier to read
plt.figure(figsize=(10,10)) # Change graph size
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by County')
plt.xlabel('Count')
plt.ylabel('County')

# Horizontal Bar Plot so that county names fit
records_df['county_cleaned'].value_counts().plot.barh();

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 5:</b>
    <br />
    How many counties have only one record of <i>Argia agrioides</i>? 
</div>

#student

YOUR RESPONSE HERE

Here's an example of Pandas code that can calculate it for you. It's kind of confusing so don't sweat the details. Compare this with your answer above.

In [ ]:
sum(records_df['county_cleaned'].value_counts() == 1)

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 6:</b>
    <br />
    Stanislaus County has the highest record of <i>Argia agrioides</i>. Other than high abundance in this county, why else might there be a high number of records here?
</div>

#student

YOUR RESPONSE HERE

Let's take a look at how each record was obtained.

In [ ]:
records_df['basisOfRecord'].value_counts().plot.bar();

# This makes the graph easier to read
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by Source');
plt.xticks(rotation=0);
plt.xlabel('Source');
plt.ylabel('Count');

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 7:</b>
    <br />
    What are some cautions that should be taken when including human observations? What are the benefits? 
</div>

#student

YOUR RESPONSE HERE

In [ ]:
records_df['collectionCode'].value_counts().plot.barh();

# This makes the graph easier to read
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by Collection');
plt.xlabel('Collection');
plt.ylabel('Count');

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 8:</b>
    <br />
    Each museum has a unique institution code (called a collection code). How many records belong to the Essig Museum of Entomology Collection?
</div>

#student

YOUR RESPONSE HERE

It's harder to check the exact number of records this time because the scale is larger. Let's check how close your answer was to the true amount.

In [ ]:
sum(records_df['collectionCode'] == 'EMEC')

---

The `groupby()` method allows us to group our data by the values in a specified column. We can then graph our data.

In [ ]:
records_df.groupby(["collectionCode", "basisOfRecord"])['basisOfRecord'].count().unstack().plot.barh(stacked=True);

# This makes the graph easier to read
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by Collection Code and Source');
plt.xticks(rotation=0);
plt.xlabel('Count');
plt.ylabel('Collection');

Here's another example using County and Collection Code.

In [ ]:
records_df.groupby(["county_cleaned", "collectionCode"])['collectionCode'].count().unstack().plot.barh(stacked=True, figsize=(10, 10));

# Make graph easier to read
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by County and Collection');
plt.xlabel('Count');
plt.ylabel('County');

And we can use `plot.hist()` to make a histogram:

In [ ]:
records_df['elevation'].plot.hist();

# Make graph easier to read
plt.title(r"${Argia}$" + r" ${agrioides}$" + ' Distribution by Elevation')
plt.xlabel('Elevation (m)');

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 9:</b>
    <br />
    What does plotting the elevation indicate about the distribution of <i>Argia agrioides</i>? Can you infer anything about the biology of the organism from this information?
</div>

#student

YOUR RESPONSE HERE

---

<div class="alert alert-block alert-info">
    <b>EXERCISE 1:</b>
    <br />
    Edit the code below to search for a different, non-domesticated species you're interested in, then use the graphing cells below to explore your data! Pick a species and replace ... with its scientific name. Make sure to add quotation marks around the name!
</div>

**Hint:** Here's what the code looks like if we used _Argia agrioides_ again:
```
my_records_df = get_species_records_df('Argia agrioides')
my_records_df.head()
```

In [ ]:
# Replace the ... in the line below with your species
my_records_df = get_species_records_df(...)
my_records_df.head() # Show the first 5 records

If the output above doesn't contain a table, that means either you didn't enter a proper scientific name or the scientific name isn't in the database. Make sure you typed it correctly without abbreviating the species name. You might also have to use a different capitalization.

In [ ]:
my_records_df['year'].plot.hist();

# Make graph easier to read
plt.title('Species Distribution by Year')
plt.xlabel('Year');

In [ ]:
# Alternative plot to use if there isn't a collection code
# my_records_df['county'].value_counts().plot.barh();

my_records_df['county_cleaned'] = my_records_df['county'].apply(clean_up_county)
my_records_df.groupby(["county_cleaned", "collectionCode"])['collectionCode'].count().unstack().plot.barh(stacked=True, figsize=(10, 10));

# Make graph easier to read
plt.title('Species Distribution by County and Collection');
plt.xlabel('Count');
plt.ylabel('County');

<b>Note: </b> If the species you chose is distributed globally, the above plot may look a bit odd or the cell may have errored. Uncomment the cell below (you can highlight the entire cell and enter Control + / on PC or Command + / on Mac on your keyboard) and run it to see the same chart but with countries instead of counties.

In [ ]:
# my_records_df.groupby(["country", "collectionCode"])['collectionCode'].count().unstack().plot.barh(stacked=True, figsize=(10, 10));
# plt.title('Species Distribution by Country and Collection');
# plt.xlabel('Count');
# plt.ylabel('Country');

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 10:</b>
    <br />
    What county has the highest number records? If you chose a globally distributed species, what country has the highest number of records?
</div>

#student

YOUR RESPONSE HERE

In [ ]:
my_records_df['elevation'].plot.hist();

# Make graph easier to read
plt.title('Species Distribution by Elevation')
plt.xlabel('Elevation (m)');

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 11:</b>
    <br />
    What is the elevation range of your organism?
    <br/>
    <i>Note:</i> Depending on the species you have chosen, your data frame might not have any values to plot. If this is the case for your species, why might this be?
</div>

#student

YOUR RESPONSE HERE

In [ ]:
my_records_df['basisOfRecord'].value_counts().plot.bar();

# This makes the graph easier to read
plt.title('Species Distribution by Source');
plt.xticks(rotation=0);
plt.xlabel('Source');
plt.ylabel('Count');

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 12:</b>
    <br />
    Which has more: observations or preserved specimens? Why might this be?
</div>

#student

YOUR RESPONSE HERE

# Part 3: Mapping  <a id='mapping'></a>

In the rest of this notebook, we will be using functions we have defined ourselves in order to reduce complex and repetitive code. Writing functions like this allows us to ignore the details and structure of how functions work and focus on the results of the code. Just remember that a **function** is a shortcut to easily re-run old code and that the `def` keyword means we are creating a function.

These functions get the species record from the API (like in Part 2). The function `get_species_record` gives us the raw/unorganized records, while `get_species_records_df` gives us the data in a **DataFrame** (table of data). It uses the same commands as we used in Part 2.

In [ ]:
def get_species_records(scientific_name):
    req = GBIFRequest()  # creating a request to the API
    params = {'scientificName': scientific_name}  # setting our parameters (the specific species we want)
    pages = req.get_pages(params)  # using those parameters to complete the request
    records = [rec for page in pages for rec in page['results'] if rec.get('decimalLatitude')]  # sift out valid records
    return records
    
def get_species_records_df(scientific_name):
    records = get_species_records(scientific_name) # Get the records using the function above
    records_df = pd.read_json(json.dumps(records)) # Convert the raw records into a DataFrame
    return records_df

This creates the **DataFrame** (table of data) we used in Part 2 using one of the functions we defined in the cell above. It will take a few seconds to get the records from the API.

In [ ]:
argia_agrioides_df = get_species_records_df('Argia agrioides')
argia_agrioides_df.head() # Show the first 5 records

Since we are about to map all of the _Argia agrioides_ specimen by their collection, let's assign each collection a color. These colors are chosen randomly each time the cell is run so you can re-run the cell if you don't like them.

In [ ]:
color_dict, html_key = assign_colors(argia_agrioides_df, 'collectionCode')
display(HTML(html_key))

Folium is a useful library for generating map visualizations. Here, we create a function that handles the Folium mapping for us.

In [ ]:
# This function generates a map visualization using data from species_df and child (if a value is given)
# Grouping Criteria tells Folium how to group specimen by color (ex. by collection or by species)
# Child is any secondary data we want to display (ex. UC Reserve boundaries)
def map_species_with_folium(species_df, grouping_criteria, child=None):
    map = folium.Map(location=[37.359276, -122.179626], zoom_start=5) # Creates the starting map location & zoom
    if child: # If a child is given, add it to the map
        map.add_child(child)
    for r in species_df.iterrows(): # For ever specimen in the species record, do the following:
        lat, long = r[1]['decimalLatitude'], r[1]['decimalLongitude'] # Get the specimen latitude/longitude
        # Add the specimen to the map
        folium.CircleMarker((lat, long), color=color_dict[r[1][grouping_criteria]]).add_to(map)
    return map

Let's map the _Argia agrioides_ specimen distribution using the function we just created.

In [ ]:
argia_agrioides_map = map_species_with_folium(argia_agrioides_df, 'collectionCode')
argia_agrioides_map

---

Let's map the distribution of _Argia agrioides_ with the boundaries of UC Reserves.

To get the boundaries for all the reserves, we will need to send a request to get GeoJSON, which is a format for encoding a variety of geographic data structures. With this code, we can request GeoJSON for all reserves and plot ocurrences of the species.

First, we'll assign the API URL that has the data to a new variable `url`. Then, we make the requests just like we did earlier through the GBIF. You'll see a huge mess of mostly numbers. This is a JSON of all the UC Reserves and the coordinates of their boundaries.

In [ ]:
url = 'https://ecoengine.berkeley.edu/api/layers/reserves/features/'
reserves = requests.get(url, params={'page_size': 30}).json()
reserves

There are some reserves that the EcoEngine didn't catch. We'll add the information for "Blodgett", "Hopland", and "Sagehen" manually.

In [ ]:
station_urls = {
    'Blodgett Reserve': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/BlodgettForestResearchStation.wkt',
    'Hopland Reserve': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/HoplandResearchAndExtensionCenter.wkt',
    'Sagehen Reserve': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/SagehenCreekFieldStation.wkt'
}
reserves['features'] += [{'type': 'Feature', 'properties': {'name': name}, 'geometry':
                          mapping(wkt.loads(requests.get(url).text))} for name, url in station_urls.items()]

This code goes through our list of reserves and outputs their names. Make sure "Blodgett", "Hopland", and "Sagehen" are included!

In [ ]:
[r['properties']['name'] for r in reserves['features']]

We can send this `geojson` directly to our mapping library `folium`. We already defined a function to do this for us, so the code is much shorter. You'll have to zoom in, but you should see blue outlined areas. Those are the reserves!

In [ ]:
reserve_points = folium.features.GeoJson(reserves) # This tells Folium our reserve boundaries
argia_agrioides_and_reserves_map = map_species_with_folium(argia_agrioides_df, 'collectionCode', child=reserve_points)
argia_agrioides_and_reserves_map

**To answer the question, double click on YOUR RESPONSE HERE. Then run the cell afterwards.**

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 13:</b>
    <br />
    The UC Reserves are a tremendous resource for researchers and students. You can zoom in to make the reserve boundaries more visible and see the geographic characteristics of each reserve. 
    <br />
    Find one reserve where <i>A. agrioides</i> was collected. Do the characteristics of the reserve fit with what you know about the biology of <i>Agria agrioides</i> (mainly lower elevation, riparian zone). Is there another reserve that also seems like it is a suitable habitat?
</div>

#student

YOUR RESPONSE HERE

---

Now that we've mapped the _Agria agrioides_ specimen, let's do that with a different species.

<div class="alert alert-block alert-info">
    <b>EXERCISE 2:</b>
    <br />
    Pick a species and replace ... with its scientific name. Make sure to add quotation marks around the name!
</div>

**Hint:** Here's what the code looks like if we used _Argia agrioides_ again:
```
my_species_df = get_species_records_df('Argia agrioides')
my_species_df.head()
```

In [ ]:
# Replace the ... in the line below with your species name
my_species_df = get_species_records_df(...)
my_species_df.head() # Show the first 5 records

If the output above doesn't contain a table, that means either you didn't enter a name or the scientific name isn't in the database. Make sure you typed it correctly without abbreviating the species name. You might also have to use a different capitalization.

<div class="alert alert-block alert-info">
    <b>EXERCISE 3:</b>
    <br />
    Assign colors to each collection by replacing ... with the name of the DataFrame we just created (my_species_df). Make sure you <b>don't</b> add quotation marks this time! Also, be careful to not accidentally delete the comma!
</div>

**Hint:** Here's what the code looks like with the `argia_agrioides_df` DataFrame:
```
color_dict, html_key = assign_colors(argia_agrioides_df, 'collectionCode')
display(HTML(html_key))
```

In [ ]:
# Replace the ... in the line below with my_species_df
# If you changed the name of the variable in the code cell above, use that name instead
color_dict, html_key = assign_colors(..., 'collectionCode')
display(HTML(html_key))

Let's map your species with Folium!

<div class="alert alert-block alert-info">
    <b>EXERCISE 4:</b>
    <br />
    Now let's map your species. Replace ... with the name of the DataFrame we just created (my_species_df). Make sure you <b>don't</b> add quotation marks this time! Also, be careful to not accidentally delete the comma!
</div>

**Hint:** Here's what the code looks like with the `argia_agrioides_df` DataFrame:
```
reserve_points = folium.features.GeoJson(reserves) # Adds reserve boundaries
my_species_map = map_species_with_folium(argia_agrioides_df, 'collectionCode', child=reserve_points)
my_species_map
```

In [ ]:
reserve_points = folium.features.GeoJson(reserves) # Adds reserve boundaries

# Replace the ... in the line below
my_species_map = map_species_with_folium(..., 'collectionCode', child=reserve_points)
my_species_map

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 14:</b>
    <br />
    Make some inferences about the biology of your mapped organism from the mapped distribution. Consider the end of the species range. What conditions (both biotic and abiotic) might be limiting the range?
</div>

#student

YOUR RESPONSE HERE

---

Now let's go back to looking at _Argia argrioides_.

We can also find out which stations have how many _Argia argrioides_. First we'll have to add a column to our DataFrame that makes points out of the latitude and longitude coordinates.

In [ ]:
station_df = argia_agrioides_df

def make_point(row):
    return Point(row['decimalLongitude'], row['decimalLatitude'])

station_df['point'] = station_df.apply(lambda row: make_point (row), axis=1)

Now we can write a little function to check whether that point is in one of the stations, and if it is, we'll add that station in a new column called `station`. Then we'll apply that function the DataFrame.

In [ ]:
def in_station(reserves, row):
    reserve_polygons = []
    for r in reserves['features']:
        name, poly = r['properties']['name'], sg.shape(r['geometry'])
        reserve_polygons.append({'id': name, 'geometry': poly})
    sid = False
    for r in reserve_polygons:
        if r['geometry'].contains(row['point']):
            sid = r['id']
    return sid

station_df['station'] = station_df.apply(lambda row: in_station(reserves, row),axis=1)
in_stations_df = station_df[station_df['station'] != False]
in_stations_df.head()

Let's see if this corresponds to what we observed on the map:

In [ ]:
in_stations_df.groupby(['species', 'station'])['station'].count().unstack().plot.barh(stacked=True);

---

# Part 4: Comparing California Oak Species  <a id='oak'></a>

Instead of investigating just one species, let’s compare several different species.

California oaks are common woody plants across North America. Almost all oaks are trees, but in drier areas they can be found as shrub oaks in poorer soils. Thus, they can be a great model system to illuminate the processes of speciation, adaptation, and expression. Let’s see their distribution!

<table style='center'>
  <tr>
    <td style="text-align: center; vertical-align: middle;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Large_Blue_Oak.jpg/220px-Large_Blue_Oak.jpg" alt="Quercus douglassi" />
        <br />
        Quercus douglassi
    </td>
    <td style="text-align: center; vertical-align: middle;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/86/Valley_Oak_Mount_Diablo.jpg/220px-Valley_Oak_Mount_Diablo.jpg" alt="Quercus lobata" />
        <br />
        Quercus lobata
    </td>
  </tr>
  <tr>
    <td style="text-align: center; vertical-align: middle;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/Quercusduratadurata.jpg/220px-Quercusduratadurata.jpg" alt="Quercus durata" />
        <br />
        Quercus durata
    </td>
    <td style="text-align: center; vertical-align: middle;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Quercus_agrifolia_foliage.jpg/220px-Quercus_agrifolia_foliage.jpg" alt="Quercus agrifolia" />
        <br />
        Quercus agrifolia
    </td>
  </tr>
</table>

Let's get the California oak records. This cell will take a while to run (about 30 seconds).

In [ ]:
species_dfs = []
species = ['Quercus douglassi', 'Quercus lobata', 'Quercus durata', 'Quercus agrifolia']

# Here, we're getting the species record from the API for the four species of oak trees listed above
for s in species:
    species_dfs.append(get_species_records_df(s))

# Combine the data we received into one DataFrame
oak_df = pd.concat(species_dfs, axis=0, sort=True)

In [ ]:
oak_df.head() # Show the first 5 rows of our data

The table above only shows us the first 5 rows. Run the cell below to see how many total records we have.

In [ ]:
len(oak_df)

Let's see how those records are distributed by species.

In [ ]:
# This creates a bar graph showing the distribution of species in our records
oak_df['scientificName'].value_counts().plot.barh();

We can also map these like we did with the *Argia arioides* above:

In [ ]:
color_dict, html_key = assign_colors(oak_df, 'scientificName')
display(HTML(html_key))

In [ ]:
oak_map = map_species_with_folium(oak_df, 'scientificName', child=folium.features.GeoJson(reserves))
oak_map

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 15:</b>
    <br />
    Examine the map you generated of <i>Quercus spp</i>. In some places the geographic range of each species overlaps and in other parts of California the range is non-overlapping. Discuss factors that create this patterning in the oak community. Include concepts of niche and competitive exclusion.
</div>

#student

YOUR RESPONSE HERE

---

# Part 5: Cal-Adapt  <a id='caladapt'></a>

Let's go back to the data from _Argia agrioides_ with the GBIF API. This will take a few seconds. The output is also really long. Remember you can click the area to the left of the cell (below the red `Out[ ]`) to expand/collapse the output.

In [ ]:
# Get the first five records in raw text form (rather than DataFrame)
argia_agrioides_records = get_species_records('Argia agrioides')
argia_agrioides_records[:5] # Show the first 5 records
# This looks different from the records from our earlier records because these are the raw records

Now we will use the [Cal-Adapt](http://www.cal-adapt.org/) Web API to work with time series raster data. It will request an entire time series for any geometry and return a DataFrame for each record in all of our _Argia agrioides_ records. This cell also takes a while to run (1-3 minutes).

In [ ]:
req = CalAdaptRequest()
record_geometry = [dict(rec, geometry=sg.Point(rec['decimalLongitude'], rec['decimalLatitude']))
             for rec in argia_agrioides_records]
ca_df = req.concat_features(record_geometry, 'gbifID')
ca_df.head() # Show the first five rows

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 16:</b>
    <br />
    What is Cal-Adapt? What can it be used for?
</div>

#student

YOUR RESPONSE HERE

This looks like the time series data we want for each record (the unique ID numbers as the columns). Each record has the projected temperature in Fahrenheit for 170 years (every row!). We can plot predictions for few random records:

In [ ]:
# Make a line plot using the first 9 columns of df.
ca_df.iloc[:,:9].plot();

# Use matplotlib to title your plot.
plt.title('Argia agrioides - %s' % req.slug)

# Use matplotlib to add labels to the x and y axes of your plot.
plt.xlabel('Year', fontsize=18)
plt.ylabel('Degrees (Fahrenheit)', fontsize=16);

It looks like temperature is increasing across the board wherever these observations are occuring. We can calculate the average temperature for each year across observations in California:

In [ ]:
tmax_means = ca_df.mean(axis=1)
# This is just some Pandas code to make the data prettier
pd.DataFrame(tmax_means).reset_index().rename(columns={'event':'Year', 0:'Avg Projected Temp'})

What's happening to the average temperature that *Argia agrioides* is going to experience in the coming years across California?

In [ ]:
tmax_means.plot()
plt.title("Average Projected Temperature")
plt.ylabel("Degrees (Fahrenheit)")
plt.xlabel("Year");

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 17:</b>
    <br/>
    Based on the plot of temperature projections above and considering the range and ecology of <i>Argia agrioides</i>, what is the expected impact on <i>A. agrioides</i>? Consider the thermal tolerance range of <i>A. agrioides</i> and its ideal temperature range. 
</div>

<b>Hint: </b> You do not need to do any outside research for this question. <i>Agria agrioides</i> occur at shallow sandy and rocky streams in the open, with moderate current and beds of vegetation such as watercress. You should be able to construct a thoughtful answer about the expected impact on <i>A. agrioides</i> with just this information and a bit of inference.

#student

YOUR RESPONSE HERE

#student
<div class="alert alert-block alert-warning">
    <b>QUESTION 18:</b>
    <br />
    What does this tell you about Santa Cruz Island? As time goes on and the temperature increases, might Santa Cruz Island serve as a refuge for <i>Argia agrioides</i>?
</div>

#student

YOUR RESPONSE HERE

---

**Make sure that you've answered questions 1-18. Also make sure you've done the 4 code exercises.**

You are finished with this notebook! Please run the following cell to generate a download link for your submission file to submit on bCourses. ***It is very likely that this download link will not work. If the download link does not work, please use the alternate download method described below.***

Alternate download instructions:
- open a new tab and go to https://datahub.berkeley.edu
- go to the "fall2019" folder
- click the box next to `museums_module_submission.pdf`
- click the "Download" link below the menu bar

**Check the PDF before submitting and make sure all of your answers & code changes are shown.**

In [ ]:
!pip install gsexport -q
import gsExport
gsExport.generateSubmission("museums_module.ipynb")

---

Notebook developed by: Michelle Koo, Nina Pak, Natalie Graham, Monica Wilkinson, Andy Sheu, Harry Li

[Data Science Modules](http://data.berkeley.edu/education/modules)